### CNN

In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [4]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [5]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [6]:
X_train.shape

(60000, 28, 28)

In [7]:
tf.random.set_seed(42)

In [8]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# y_train을 one-hot 인코딩
y_train = to_categorical(y_train, num_classes=10)

# 모델을 생성하는 함수 정의
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, input_shape=(28,28,1), padding='same', activation='relu', kernel_initializer="he_normal"),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=10, activation='softmax')
    ])

    # 모델 컴파일
    model.compile(optimizer='Adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


    return model


In [ ]:
import numpy as np

# 콜백 설정
call_back = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.0001)

k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)

    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate([X_train[:i * num_val], X_train[(i + 1) * num_val:]], axis=0)
    y_train_part = np.concatenate([y_train[:i * num_val], y_train[(i + 1) * num_val:]], axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part, epochs=num_epochs, verbose=1, callbacks=[call_back])

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_acc)


processing fold # 0
Epoch 1/100
1688/1688 [==============================] - ETA: 0s - loss: 0.4425 - accuracy: 0.8391

1688/1688 [==============================] - 16s 6ms/step - loss: 0.4425 - accuracy: 0.8391 - lr: 0.0010
Epoch 2/100
1688/1688 [==============================] - ETA: 0s - loss: 0.2785 - accuracy: 0.8989

1688/1688 [==============================] - 10s 6ms/step - loss: 0.2785 - accuracy: 0.8989 - lr: 0.0010
Epoch 3/100
1679/1688 [============================>.] - ETA: 0s - loss: 0.2365 - accuracy: 0.9138

1688/1688 [==============================] - 10s 6ms/step - loss: 0.2367 - accuracy: 0.9137 - lr: 0.0010
Epoch 4/100
1684/1688 [============================>.] - ETA: 0s - loss: 0.2078 - accuracy: 0.9234

1688/1688 [==============================] - 11s 6ms/step - loss: 0.2078 - accuracy: 0.9234 - lr: 0.0010
Epoch 5/100
1686/1688 [============================>.] - ETA: 0s - loss: 0.1865 - accuracy: 0.9310

...

1688/1688 [==============================] - 11s 7ms/step - loss: 0.0411 - accuracy: 0.9871 - lr: 0.0010
Epoch 66/100
1686/1688 [============================>.] - ETA: 0s - loss: 0.0417 - accuracy: 0.9877

1688/1688 [==============================] - 10s 6ms/step - loss: 0.0417 - accuracy: 0.9877 - lr: 0.0010
Epoch 67/100
1686/1688 [============================>.] - ETA: 0s - loss: 0.0435 - accuracy: 0.9871

1688/1688 [==============================] - 10s 6ms/step - loss: 0.0434 - accuracy: 0.9871 - lr: 0.0010
Epoch 68/100
1680/1688 [============================>.] - ETA: 0s - loss: 0.0404 - accuracy: 0.9873

1688/1688 [==============================] - 10s 6ms/step - loss: 0.0404 - accuracy: 0.9872 - lr: 0.0010
Epoch 69/100
1679/1688 [============================>.] - ETA: 0s - loss: 0.0454 - accuracy: 0.9875

1688/1688 [==============================] - 10s 6ms/step - loss: 0.0456 - accuracy: 0.9874 - lr: 0.0010
Epoch 70/100
 380/1688 [=====>........................] - ETA: 7s - loss: 0.0402 - accuracy: 0.9887

In [ ]:
all_scores

In [ ]:
np.mean(all_scores)

### 결과 1

In [ ]:
all_scores

# from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.utils import to_categorical

# # y_train을 one-hot 인코딩
# y_train = to_categorical(y_train, num_classes=10)

# # 모델을 생성하는 함수 정의
# def build_model():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, input_shape=(28,28,1), padding='same', activation='relu', kernel_initializer="he_normal"),
#         tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
#         tf.keras.layers.MaxPool2D(pool_size=(2,2)),
#         tf.keras.layers.Dropout(0.2),

#         tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
#         tf.keras.layers.MaxPool2D(pool_size=(2,2)),
#         tf.keras.layers.Dropout(0.2),

#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(units=256, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(units=10, activation='softmax')
#     ])

#     # 모델 컴파일
#     model.compile(optimizer='Adam',
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])


#     return model


[0.934333324432373,
 0.9325000047683716,
 0.9259999990463257,
 0.9353333115577698,
 0.9365000128746033,
 0.9325000047683716,
 0.9316666722297668,
 0.9390000104904175,
 0.9366666674613953,
 0.9334999918937683]

In [ ]:
np.mean(all_scores)

0.9337999999523163

### 결과 1이 0.9337999999523163로 제일 높음